### Kaggle_Movie recommand system

- 컨텐츠 필터링 기반 추천시스템(장르, 배우, 예산 등..)

In [2]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('data/movies_metadata.csv')

In [8]:
data.head()
# data.shape # (45466, 24)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [12]:
data = data[['id','genres','vote_average', 'vote_count','popularity', 'title','overview']]

In [14]:
# Q. How do you calculate the rank of movies and TV shows on the Top 250 Rated Charts?
# A. weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
    # R = average for the movie (mean) = (Rating)
    # v = number of votes for the movie = (votes)
    # m = minimum votes required to be listed in the Top 250 (currently 25,000)
    # C = the mean vote across the whole report

In [17]:
data['vote_count'].quantile(0.7) # 70% 지점에 해당하는 값  # quantile(백분위수)

25.0

In [20]:
data['vote_count'].max() # 14075
data['vote_count'].min() # 0

0.0

In [25]:
tmp_m = data['vote_count'].quantile(0.95) #434  # vote_count값이 상위 5%내에 들기 위한 최솟값

In [26]:
cdata = data.copy()

In [27]:
cdata.shape

(45466, 7)

In [29]:
tmp_data = cdata.loc[cdata['vote_count']>=tmp_m]
tmp_data

,id,genres,vote_average,vote_count,popularity,title,overview
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.9469,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.0155,Jumanji,When siblings Judy and Peter discover an encha...
5,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",7.7,1886.0,17.9249,Heat,"Obsessive master thief, Neil McCauley leads a ..."
9,710,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",6.6,1194.0,14.686,GoldenEye,James Bond must unmask the mysterious head of ...
15,524,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",7.8,1343.0,10.1374,Casino,The life of the gambling paradise – Las Vegas ...
...,...,...,...,...,...,...,...
44624,406990,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",7.3,598.0,60.581223,What Happened to Monday,In a world where families are limited to one c...
44632,341013,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",6.1,748.0,14.455104,Atomic Blonde,An undercover MI6 agent is sent to Berlin duri...
44678,374720,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",7.5,2712.0,30.938854,Dunkirk,The miraculous evacuation of Allied soldiers f...
44842,335988,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",6.2,1440.0,39.186819,Transformers: The Last Knight,"Autobots and Decepticons are at war, with huma..."


In [33]:
C = tmp_data['vote_average'].mean()  # 최소 평균 평점: 6.6
M = data['vote_count'].quantile(0.95)  # 434

def weighted_rating(x, m=m, c=c):
    